In [ ]:
# Imports for JupyterLite
%pip install -q ipywidgets requests py-cptcore

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pyodide_http

from pycptcore import SoilProperties
import requests
import json

pyodide_http.patch_all()

In [ ]:
file = widgets.FileUpload(
    accept=".gef, .GEF",  # Accepted file extension
    multiple=False,  # True to accept multiple files upload else False
    disabled=False,
)
authorization = widgets.Password(
    value=None,
    placeholder="Enter Personal Access Token",
    description="Authorization:",
    disabled=False,
)
classify = widgets.Dropdown(
    options=["beenJefferies", "machineLearning", "nen", "table", "robertson", "table"],
    value="nen",
    description="Classification methode:",
    disabled=False,
)

In [ ]:
display(file)
display(authorization)

In [ ]:
if not len(file.value) > 0:
    raise ValueError("Please provide a file first!")

# create payload for parse cpt call
payload = {
    "content": file.value[0].content.tobytes().decode(encoding="iso8859-1"),
    "engine": "gef",
}

# call methode
parse_response = requests.post(
    "https://crux-nuclei.com/api/cptcore/v1/parse/cpt",
    json=payload,
    headers={
        "Authorization": f"Bearer {authorization.value}",
    },
)

if not parse_response.ok:
    raise RuntimeError(parse_response.content)

# create soil object
soil = SoilProperties.from_api_response(parse_response.json())

In [ ]:
display(classify)

In [ ]:
# create payload for classify call
payload = {
    "aggregateLayersPenalty": 12,
    "data": {
        "coneResistance": soil.cpt_table.dataframe["coneResistance"].tolist(),
        "correctedPenetrationLength": soil.cpt_table.dataframe[
            "penetrationLength"
        ].tolist(),
    },
    "verticalPositionOffset": soil.verticalPositionOffset,
    "x": float(soil.location.transform["x"]),
    "y": float(soil.location.transform["y"]),
}

if "localFriction" in soil.cpt_table.dataframe.columns:
    payload["data"]["localFriction"] = soil.cpt_table.dataframe[
        "localFriction"
    ].tolist()

# call methode
classify_response = requests.post(
    f"https://crux-nuclei.com/api/cptcore/v1/classify/{classify.value}",
    json=payload,
    headers={
        "Authorization": f"Bearer {authorization.value}",
    },
)

if not classify_response.ok:
    raise RuntimeError(classify_response.content)

# add classify table to soil object
soil.set_layer_table_from_api_response(classify_response.json())

# create plot
figure = soil.plot()